<a href="https://colab.research.google.com/github/abhijadhav14/Deep-Learning-CSE5019/blob/main/KDD_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, BatchNormalization,
                                     Dropout, Flatten, Dense)
from tensorflow.keras.utils import to_categorical

In [2]:
url_train = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
url_test = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

In [3]:
columns = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
           "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
           "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login",
           "is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate",
           "srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
           "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
           "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
           "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

In [4]:
train_df = pd.read_csv(url_train, names=columns)
test_df = pd.read_csv(url_test, names=columns)

In [5]:
categorical = ['protocol_type', 'service', 'flag']
encoder = LabelEncoder()
for col in categorical:
    train_df[col] = encoder.fit_transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])

In [6]:
X_train = train_df.drop(['label', 'difficulty'], axis=1)
y_train = train_df['label']
X_test = test_df.drop(['label', 'difficulty'], axis=1)
y_test = test_df['label']

In [7]:
y_train = y_train.apply(lambda x: 0 if x == 'normal' else 1)
y_test = y_test.apply(lambda x: 0 if x == 'normal' else 1)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
X_train = X_train.reshape(X_train.shape[0], 41, 1, 1)
X_test = X_test.reshape(X_test.shape[0], 41, 1, 1)

In [10]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
model = Sequential()

model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu', input_shape=(41,1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=1)

Epoch 1/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 27s 28ms/step - accuracy: 0.9369 - loss: 0.2073 - val_accuracy: 0.9868 - val_loss: 0.0389
Epoch 2/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.9770 - loss: 0.0606 - val_accuracy: 0.9901 - val_loss: 0.0304
Epoch 3/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 38s 25ms/step - accuracy: 0.9824 - loss: 0.0465 - val_accuracy: 0.9906 - val_loss: 0.0281
Epoch 4/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.9835 - loss: 0.0431 - val_accuracy: 0.9919 - val_loss: 0.0249
Epoch 5/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9856 - loss: 0.0372 - val_accuracy: 0.9910 - val_loss: 0.0252
Epoch 6/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9870 - loss: 0.0351 - val_accuracy: 0.9929 - val_loss: 0.0225
Epoch 7/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.9865 - loss: 0.0351 - val_accuracy: 0.9927 - val_loss: 0.0206
Epoch 8/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9884 - loss: 0.0317 - 

In [17]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nTest Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

705/705 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

Test Accuracy: 0.7865063875088716

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.98      0.80      9711
           1       0.97      0.64      0.77     12833

    accuracy                           0.79     22544
   macro avg       0.82      0.81      0.79     22544
weighted avg       0.84      0.79      0.78     22544

